# Assignment 4.  




## Queries combining multiple tables

Read chapters 7-11

In this assignment, you will work on "Problems for you to Solve" from Chapters 8, 9, and 11 in *SQL Queries for Mere Mortals*. 

We will use DataJoint to solve these problems and then review the same solutions in SQL.




# Queries with DataJoint

## Connect to the database server


In [ ]:
import datajoint as dj

In [ ]:
# Execute this only if you have not connected with datajoint before.  
import json
with open('cred.json') as f:
    creds = json.load(f)

dj.config['database.host'] = creds['host']
dj.config['database.user'] = creds['user']
dj.config['database.password'] = creds['password']

dj.config.save_local()

In [ ]:
dj.list_schemas()

In [ ]:
schema = dj.schema('shared_dj_sales')
schema.spawn_missing_classes()

In [ ]:
dj.Diagram(schema)

If you could not see the graph, then go to the DataJoint documentation to install `pydotplus`  https://docs.datajoint.io/python/setup/02-DataJoint-Python-Windows-Install-Guide.html

## Simple Query

Customers()

## Fetching

In [ ]:
arr = Customers().fetch()  # retrieve as a numpy recarray

In [ ]:
dicts = Customers().fetch(as_dict=True)  # retrieve as a list of dictionaries

In [ ]:
df = Customers().fetch(format='frame')  # retrieve as a pandas dataframe

## Restriction

In [ ]:
Customers() & {'cust_state': 'WA'}  # restrict by a dict

In [ ]:
Customers() & 'cust_state ="WA"'   # restrict by string

In [ ]:
Customers() & {'customer_id': 1001}

In [ ]:
keys = Customers.fetch('KEY')

In [ ]:
Customers & keys[2]

In [ ]:
# Give me all the products that cost more than $1000
Products() & 'retail_price > 1000'

## Negative restriction
Customers who are not from WA

In [ ]:
Customers() - {'cust_state': 'WA'}

## Projection 

In [ ]:
# always includes primary key
Customers.proj()

In [ ]:
q = Customers.proj('cust_last_name', 'cust_first_name')

In [ ]:
q

In [ ]:
Products()

In [ ]:
q = Products().proj('product_name', 'retail_price', 
                    stock_value='retail_price * quantity_on_hand')
q 

In [ ]:
# give me all products whose stock value is over 5000
(q & 'stock_value > 5000').proj('stock_value')

## Restricting with another query 


In [ ]:
dj.Diagram(schema)

In [ ]:
# All customers who have made an order

Customers & Orders

In [ ]:
# All customers who have not made an order

Customers - Orders

In [ ]:
(Customers & Orders).make_sql()

In [ ]:
# Give me all orders for items whose price exceeds $1000

In [ ]:
expensive_products = Products & 'retail_price > 1000'
big_orders = Orders & (OrderDetails & expensive_products)
big_orders

In [ ]:
big_orders.make_sql()

In [ ]:
# Show all products from the category "Bikes"
bikes = Categories() & 'category_description = "Bikes"'
Products() & bikes

In [ ]:
(Products & bikes).make_sql()

In [ ]:
# All order IDs where the customer was from Washington
washington_customers = Customers & 'cust_state="WA"'
(Orders & washington_customers).proj()

## Joins 

In [ ]:
Products * Categories

In [ ]:
(Products * Categories & 'category_description = "Bikes"').proj('product_name', 'retail_price')

In [ ]:
(Products & (Categories & 'category_description = "Bikes"')).proj('product_name', 'retail_price')

In [ ]:
# All the orders that took more than 2 days to ship
Orders().proj(days_to_ship='DATEDIFF(ship_date, order_date)') & 'days_to_ship > 2'

In [ ]:
ProductVendors()

In [ ]:
discounts = OrderDetails * Products & 'quoted_price < retail_price'
(Orders & discounts).proj('order_date')

In [ ]:
dj.Diagram(schema)

In [ ]:
# Find all orders with products that have vendors from TX
Orders & (OrderDetails * Products * ProductVendors * Vendors & 'vend_state = "TX"')

In [ ]:
# Give all products some vendors are from TX
Products & (ProductVendors * Vendors & 'vend_state="TX"')

In [ ]:
# Advanced: Give all products where all vendors are from TX
Products - (ProductVendors * Vendors - 'vend_state="TX"')

In [ ]:
# rename "retail_price" into "price"
Products().proj(..., price='retail_price')

In [ ]:
Customers() * Products()

In [ ]:
OrderDetails() * Orders()

# Homework

Write the following queries in DataJoint:

In [ ]:
# 1. List all products that need to be restocked (less than 10 are left on hand) 
# (6 items)

In [ ]:
# 2. List all orders made since "2018-01-01"
# (350 items)

In [ ]:
# 3. List customers who have never ordered anything.
# (1 item)

In [ ]:
# 4. List all orders for bicycles
# (586 items)

In [ ]:
# 5. List customers whose first name is Liz.
# (1 item)

In [ ]:
# 6. List all orders made by Liz.
# (39 items)

In [ ]:
# 7. List all orders made by employees named Susan.
# (130 items)

In [ ]:
# 8. List all employees who have not sold anything to Liz since 2018-01-01
# (1 item)

In [ ]:
# 9. List the names of all the customers to whom Susan has sold something in Sept 2017
# (17 items)

In [ ]:
# 10. List the names of products sold on 2017-09-05
# (14 items)

In [ ]:
# 11. List employees who did not make a sale on 2017-08-05
# (5 items)

In [ ]:
# 12. List empty orders (without any items)
# (11 items)

In [ ]:
# 13. List all orders where the customer and the salesperson where from the same city. 
# (41 items)

In [ ]:
# 14. List all customers who have never bought items over 1000 in quoted price 
# (5 items)